# Agregação no Apache Spark

O Apache Spark fornece uma variedade de funções para realizar operações de agregação em DataFrames. Essas operações são úteis para resumir dados, calcular estatísticas e criar janelas temporais para análises avançadas.

## Funções de Agregação

As funções de agregação permitem realizar operações em colunas de dados:

| Função                          | Descrição                                                | Exemplo de Caso de Uso                    |
|---------------------------------|----------------------------------------------------------|-------------------------------------------|
| `avg(col)`                      | Calcula a média dos valores em uma coluna                | Calcular a média de pontuações.           |
| `sum(col)`                      | Calcula a soma dos valores em uma coluna                 | Calcular a soma de vendas.                |
| `min(col)`                      | Encontra o valor mínimo em uma coluna                   | Encontrar a idade mínima dos funcionários. |
| `max(col)`                      | Encontra o valor máximo em uma coluna                   | Encontrar a idade máxima dos funcionários. |
| `count(col)`                    | Conta o número de registros não nulos em uma coluna      | Contar o número de vendas válidas.        |
| `first(col, ignorenulls=False)` | Retorna o primeiro valor não nulo em uma coluna          | Obter o primeiro registro de uma coluna.  |
| `last(col, ignorenulls=False)`  | Retorna o último valor não nulo em uma coluna            | Obter o último registro de uma coluna.    |

## Funções de Classificação (Sort Functions)

As funções de classificação permitem classificar registros em ordem crescente ou decrescente com base em uma ou mais colunas:

| Função                          | Descrição                                                | Exemplo de Caso de Uso                    |
|---------------------------------|----------------------------------------------------------|-------------------------------------------|
| `asc(col)`                      | Classifica em ordem crescente com base na coluna         | Classificar funcionários por nome.        |
| `desc(col)`                     | Classifica em ordem decrescente com base na coluna       | Classificar vendas em ordem decrescente.   |

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [3]:
df_vra = (
    spark
    .read
    .format("parquet")
    .load("/home/app/data/1.bronze/vra/")
)
df_vra.show(1, vertical=True)

-RECORD 0-----------------------------------
 icao_empresa_aerea     | TAM               
 numero_voo             | 9458              
 codigo_di              | 0                 
 codigo_tipo_linha      | I                 
 icao_aerodromo_origem  | SBGR              
 icao_aerodromo_destino | SCEL              
 partida_prevista       | 01/01/2020 06:30  
 partida_real           | 01/01/2020 06:30  
 chegada_prevista       | 01/01/2020 10:40  
 chegada_real           | 01/01/2020 10:40  
 situacao_voo           | REALIZADO         
 codigo_justificativa   | null              
 preco_passagem         | 1907.3242         
 preco_combustivel      | 19443.44365734512 
only showing top 1 row



# Group By

In [23]:
(
    df_vra
    .where(df_vra["situacao_voo"].isin(["CANCELADO","NÃO INFORMADO","REALIZADO"]))
    .groupBy(
        "icao_aerodromo_origem",
        "situacao_voo"
    )
    .agg(
        F.count("icao_aerodromo_origem").alias("count"),
        F.round(F.avg("preco_passagem"),2).alias("preco_passagem_medio")
        
    )
    .sort("count", ascending=False)
    .show()
)

+---------------------+------------+------+--------------------+
|icao_aerodromo_origem|situacao_voo| count|preco_passagem_medio|
+---------------------+------------+------+--------------------+
|                 SBGR|   REALIZADO|360724|             3966.31|
|                 SBSP|   REALIZADO|202860|             3969.64|
|                 SBKP|   REALIZADO|187684|             3962.98|
|                 SBBR|   REALIZADO|158585|             3968.81|
|                 SBRJ|   REALIZADO|138943|             3974.09|
|                 SBCF|   REALIZADO|130423|             3968.89|
|                 SBRF|   REALIZADO|112350|             3968.04|
|                 SBPA|   REALIZADO| 81326|             3978.78|
|                 SBSV|   REALIZADO| 80581|              3976.3|
|                 SBCT|   REALIZADO| 67933|              3966.2|
|                 SBGL|   REALIZADO| 65849|             3957.14|
|                 SBFZ|   REALIZADO| 58249|             3966.62|
|                 SBBE|  

# Funções de Janela (Window Functions)

Para usar as funções dessa forma, devemos criar uma janela (window) da seguinte forma:

```python
from pyspark.sql.window import Window
w = Window.partitionBy({columns}).orderBy({columns}).rowsBetween({lower}, {upper})
```
- `partitionBy()`: Refere-se ao agrupamento no qual os cálculos serão executados, sendo equivalente ao conceito de `groupBy()`.
- `orderBy`: Necessária para funções como `row_number()` e `lag()`, esta instrução determina a ordem das linhas dentro do agrupamento.
- `rowsBetween()`: Utilizado para definir janelas deslizantes, este método permite especificar um intervalo de linhas em relação à linha atual para a aplicação da função. Caso não seja especificado, as operações são aplicadas a todo o grupo, sendo especialmente útil para calcular médias móveis. Dois objetos importantes para construir esse intervalo são:
  - `Window.currentRow`: Estabelece a linha para a qual o valor está sendo calculado como um dos limites.
  - `Window.unboundedPreceding`: Indica a ausência de limites para as linhas anteriores à linha em que o valor está sendo calculado, considerando todas as linhas do grupo que já ocorreram. Deve ser utilizado no primeiro argumento (início).
  - `Window.unboundedFollowing`: Sinaliza a ausência de limites para as linhas após a linha em que o valor está sendo calculado, considerando todas as linhas do grupo que ainda não ocorreram. Deve ser utilizado no segundo argumento (fim).



As funções de janela permitem realizar cálculos em janelas temporais de dados:

| Função                          | Descrição                                                | Exemplo de Caso de Uso                    |
|---------------------------------|----------------------------------------------------------|-------------------------------------------|
| `row_number()`                  | Atribui um número de linha a cada registro na janela    | Atribuir uma classificação a registros.   |
| `rank()`                        | Atribui uma classificação aos registros na janela       | Classificar registros em uma janela.      |
| `dense_rank()`                  | Atribui uma classificação densa aos registros na janela  | Classificar registros, tratando empates.  |
| `percent_rank()`                | Calcula a classificação percentual dos registros na janela | Determinar a posição relativa dos registros. |
| `lead(col, offset, default)`    | Retorna o valor de uma coluna adiante na janela           | Analisar valores futuros em séries temporais. |
| `lag(col, offset, default)`     | Retorna o valor de uma coluna anterior na janela         | Analisar valores passados em séries temporais. |
| `first_value(col)`              | Retorna o primeiro valor de uma coluna na janela         | Obter o valor inicial em uma janela.        |
| `last_value(col)`               | Retorna o último valor de uma coluna na janela           | Obter o valor final em uma janela.          |

In [36]:
df_vra.show(1, vertical=True)

-RECORD 0-----------------------------------
 icao_empresa_aerea     | TAM               
 numero_voo             | 9458              
 codigo_di              | 0                 
 codigo_tipo_linha      | I                 
 icao_aerodromo_origem  | SBGR              
 icao_aerodromo_destino | SCEL              
 partida_prevista       | 01/01/2020 06:30  
 partida_real           | 01/01/2020 06:30  
 chegada_prevista       | 01/01/2020 10:40  
 chegada_real           | 01/01/2020 10:40  
 situacao_voo           | REALIZADO         
 codigo_justificativa   | null              
 preco_passagem         | 1907.3242         
 preco_combustivel      | 19443.44365734512 
only showing top 1 row



In [37]:
from pyspark.sql.window import Window

In [38]:
w = Window.partitionBy("icao_empresa_aerea","icao_aerodromo_origem").orderBy(F.desc('partida_prevista'))

In [45]:
(
    df_vra
    .withColumn("partida_prevista", F.to_timestamp("partida_prevista",'dd/MM/yyyy HH:mm'))
    .where(df_vra["situacao_voo"].isin(["CANCELADO","REALIZADO"]))
    .withColumn('rank', F.percent_rank().over(w))
    .select("icao_empresa_aerea","icao_aerodromo_origem",'partida_prevista','rank')
    .show(truncate=False)
)

+------------------+---------------------+-------------------+---------------------+
|icao_empresa_aerea|icao_aerodromo_origem|partida_prevista   |rank                 |
+------------------+---------------------+-------------------+---------------------+
|AAL               |KMIA                 |2023-09-30 23:53:00|0.0                  |
|AAL               |KMIA                 |2023-09-30 20:33:00|3.1969309462915604E-4|
|AAL               |KMIA                 |2023-09-30 00:10:00|6.393861892583121E-4 |
|AAL               |KMIA                 |2023-09-29 23:56:00|9.590792838874681E-4 |
|AAL               |KMIA                 |2023-09-29 21:36:00|0.0012787723785166241|
|AAL               |KMIA                 |2023-09-29 00:15:00|0.00159846547314578  |
|AAL               |KMIA                 |2023-09-28 23:55:00|0.0019181585677749361|
|AAL               |KMIA                 |2023-09-28 20:57:00|0.002237851662404092 |
|AAL               |KMIA                 |2023-09-28 00:14:00|0.0

# ...

## Nosso desafio

- Criar as seguintes views:
  ##  > Vamos fazer este  
  - Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
    - Razão social da companhia aérea
    - Nome Aeroporto de Origem
    - ICAO do aeroporto de origem
    - Estado/UF do aeroporto de origem
    - Nome do Aeroporto de Destino
    - ICAO do Aeroporto de destino
    - Estado/UF do aeroporto de destino